In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import cluster, datasets
import seaborn as sns
import json
import sys
import codecs
import os
from collections import defaultdict, Counter
import pickle

In [ ]:
#load the desired model from ./models
modeldir_path = "./models/"
model_name = "Glove50_KMeans.pkl" #enter model name here
model = pickle.load(open(modeldir_path+model_name))
print(model_name+" is loaded.")
